In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import re
import logging
import os

##  Set up logging configuration

In [2]:
import os
if not os.path.exists('logs'):
    os.makedirs('logs')

logging.basicConfig(
    filename="logs/data_cleaning.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

def log_step(step_name):
    logging.info(f"Started: {step_name}")
    
def log_error(step_name, error):
    logging.error(f"Error in {step_name}: {error}")

def log_success(step_name):
    logging.info(f"Success: {step_name}")

##  Load the CSV file into a DataFrame

In [3]:
columns_to_keep = [
    "nom",
    "prenom",
    "date_naissance",
    "cin",
    "tel",
    "email",
    "diplome",
    "etablissment",
    "formation",
    "lettre_motivation",
    "etat",
    "viewed",
    "contacte",
    "inscrit",
    "created",
    "ville",
]
log_step("Loading CSV file")
try:
    data = pd.read_csv("data.csv")
    log_success("Loading CSV file")
except Exception as e:
    log_error("Loading CSV file", e)

In [4]:

data = data[columns_to_keep]

In [5]:
data

,nom,prenom,date_naissance,cin,tel,email,diplome,etablissment,formation,lettre_motivation,etat,viewed,contacte,inscrit,created,ville
0,ER-RAKI,MERYEM,1994-07-21,G664463,672619167,meryem.er-raki@outlook.fr,bac+3,UNIVERSITE IBN TOFAI,mi,A Monsieur le directeur de l’École National De...,0,1,1,0,2015-05-20 21:11:59,NaN
1,Lihyaoui,Amine,0000-00-00,DA70851,634432230,amustube@gmail.com,bac+2,ISTA IFRANE,lar,Lihyaoui Amine 244 Bloc Elmassira 25000/khour...,0,1,1,0,2015-05-22 17:52:26,NaN
2,MEDIOUNE,Badr,1991-04-03,BB 80536,622431159,badr.medioune@gmail.com,bac+2,ISTA Sidi Moumen,lar,"Monsieur, Titulaire du diplôme technicien spéc...",0,1,1,0,2015-05-23 15:00:16,NaN
3,ELGZOULI,AMAl,1992-05-25,G634563,619957741,amale.elgzouli@gmail.com,bac+3,FST,mi,NaN,0,1,1,0,2015-05-06 00:46:07,NaN
4,belkotaine,manal,1995-07-03,ID64786,635531363,witch_girl16@hotmail.com,bac+2,ISTA NTIC Beni-Mella,lar,NaN,0,1,1,0,2015-05-07 15:40:07,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,hamcha,abdrrahman,1969-12-30,i245113,661414268,a.hamcha@africlight.ma,bac+2,ista beni mellal,MGP,bonjour,0,1,1,0,2015-10-28 13:12:27,NaN
143,mastour,amina,1993-03-21,gh679863,694950654,imdoha@gmail.com,bac+2,ista ouedzem,LRT,ddd,1,0,1,0,2015-10-28 18:07:02,NaN
144,bennah,khalil,1993-11-04,OD44737,649278532,khalilbennah@gmail.com,bac+3,faculté polydisciplinaire Errachidia,mi,"Après avoir obtenu mon ""diplôme de Licence pro...",0,0,0,0,2015-11-04 14:25:14,NaN
145,HIDARA,ADIL,1986-02-07,Q260030,615227007,hid.adil@gmail.com,bac+2,ISTA & ENMIG,lil,Très motivé pour continuer mes etude en ce dom...,0,1,0,0,2015-11-11 17:58:45,NaN


## Define validation and correction functions

### nom and prenom columns

In [6]:
def is_valid_name(value):
    if not isinstance(value, str):
        return False
    pattern = re.compile(r"^[A-Za-zÀ-ÿ -]*$")
    return bool(pattern.match(value))

def correct_name(value):
    if is_valid_name(value):
        return value.strip().title()
    return None

### date_naissance column

In [7]:
def is_valid_date(value):
    try:
        pd.to_datetime(value, format="%Y-%m-%d", errors="raise")
        return True
    except (ValueError, TypeError):
        return False


def correct_date(value):
    if is_valid_date(value):
        return value
    else:
        return None

### cin column

In [8]:
def is_valid_cin(value):
    if not isinstance(value, str):
        return False
    pattern = re.compile(r"^[A-Za-z]{1,2}\d{3,}$")
    return bool(pattern.match(value))


def correct_cin(value):
    if isinstance(value, str):
        value = value.replace(" ", "")
    if is_valid_cin(value):
        return value.upper()
    else:
        return None

### tel column

In [9]:
def is_valid_tel(value):
    if not isinstance(value, str):
        return False
    pattern = re.compile(r"^\+2126\d{8}$")
    return bool(pattern.match(value))


def correct_tel(value):
    if is_valid_tel(value):
        return value
    if not pd.isna(value):
        if not isinstance(value, str):
            value = str(int(value))
        pattern = re.compile(r"^6\d{8}$")
        if bool(pattern.match(value)) == True:
            return "+212" + value
        pattern = re.compile(r'^2126\d{8}$')
        if bool(pattern.match(value)) == True:
            return "+" + value
    return None

### email column

In [10]:
def is_valid_email(value):
    if not isinstance(value, str):
        return False
    pattern = re.compile(r"^[a-zA-Z0-9._-]+@[a-zA-Z]+\.[a-zA-Z]+$")
    return bool(pattern.match(value))


email_domains = [
    "@gmail.com",
    "@yahoo.com",
    "@outlook.com",
    "@hotmail.com",
    "@icloud.com",
    "@aol.com",
    "@protonmail.com",
    "@zoho.com",
    "@gmx.com",
    "@mail.com",
    "@yandex.com",
]


def correct_email(value):
    if is_valid_email(value):
        return value.replace(" ", "").lower()
    elif pd.isna(value) or not isinstance(value, str):
        return None
    else:
        value = value.replace(" ", "").lower()
        if value:
            if "@" in value:
                username, partial_domain = value.split("@")
                partial_domain = "@" + partial_domain
                for domain in email_domains:
                    if domain.startswith(partial_domain):
                        return username + domain
            else:
                return value + "@gmail.com"
    return None

### diplome column

In [11]:
def is_valid_diplome(value):
    if not isinstance(value, str):
        return False
    pattern = re.compile(r"^bac\+\d$")
    return bool(pattern.match(value)) or value == "autre"


def correct_diplome(value):
    if not isinstance(value, str) or pd.isna(value):
        return None
    value = value.replace(" ", "").lower()
    if is_valid_diplome(value):
        return value
    else:
        return None

### created column

In [12]:
def is_valid_created(value):
    if not isinstance(value, str):
        return False
    try:
        pd.to_datetime(value, format="%Y-%m-%d %H:%M:%S", errors="raise")
        return True
    except ValueError:
        return False


def correct_created(value):
    if is_valid_created(value):
        return value
    else:
        return None

### etat, viewed, contacte and inscrit columns

In [13]:
def is_single_digit(value):
    if not isinstance(value, str):
        return False
    pattern = re.compile(r"^\d$")
    return bool(pattern.match(value))


def correct_single_digit(value):
    if is_single_digit(value):
        return value
    else:
        return 0

### fill ville column

In [14]:
cin_city = {
    "A": "Rabat",
    "AA": "Rabat",
    "AC": "Rabat",
    "AJ": "Rabat",
    "AB": "Salé",
    "AE": "Salé",
    "AY": "Salé",
    "AS": "Salé",
    "AD": "Témara",
    "B": "Casablanca",
    "BA": "Casablanca",
    "BB": "Casablanca",
    "BE": "Casablanca",
    "BH": "Casablanca",
    "BJ": "Casablanca",
    "BK": "Casablanca",
    "BL": "Casablanca",
    "BM": "Casablanca",
    "BF": "Casablanca",
    "BV": "Casablanca",
    "BW": "Casablanca",
    "BX": "Moroccans residing abroad (MRE)",
    "DF": "Moroccans residing abroad (MRE)",
    "PK": "Moroccans residing abroad (MRE)",
    "PP": "Moroccans residing abroad (MRE)",
    "PS": "Moroccans residing abroad (MRE)",
    "PH": "Moroccans residing abroad (MRE)",
    "C": "Fez",
    "CC": "Fez",
    "CD": "Fez",
    "CB": "Sefrou",
    "CN": "Boulemane",
    "D": "Meknes",
    "DN": "Meknes",
    "DA": "Azrou",
    "DB": "Ifrane",
    "DC": "Moulay Idriss Zerhoun",
    "DJ": "Ain Taoujdate",
    "DN": "El Hajeb",
    "DO": "Ouislane",
    "E": "Marrakesh",
    "EE": "Marrakesh",
    "EA": "Ben Guerir",
    "F": "Oujda",
    "FA": "Berkane",
    "FB": "Taourirt",
    "FC": "El Aioun Sidi Mellouk",
    "FD": "Ain Bni Mathar",
    "FE": "Saïdia",
    "FG": "Figuig",
    "FH": "Jerada",
    "FJ": "Ahfir",
    "FK": "Touissit",
    "FL": "Bouarfa",
    "G": "Kenitra, Sidi Yahya El Gharb",
    "GA": "Sidi Slimane, Sidi Yahya El Gharb",
    "GB": "Souk El Arbaa",
    "GK": "Sidi Kacem",
    "GM": "Ouazzane",
    "GN": "Mechra Bel Ksiri",
    "GJ": "Jorf El Melha",
    "H": "Safi",
    "HH": "Safi",
    "HA": "Youssoufia",
    "I": "Beni Mellal",
    "IA": "Kasba Tadla",
    "IB": "Fquih Ben Saleh",
    "IC": "Azilal",
    "ID": "Souk Sebt Ould Nemma",
    "IE": "Demnate",
    "J": "Agadir",
    "JK": "Agadir",
    "JA": "Guelmim",
    "JB": "Inezgane, Dcheira El Jihadia",
    "JC": "Taroudant",
    "JD": "Sidi Ifni",
    "JE": "Tiznit",
    "JF": "Tan-Tan",
    "JH": "Chtouka Aït Baha",
    "JM": "Aït Melloul, Temsia, Lqliâa, Oulad Dahou",
    "JT": "Oulad Teima",
    "JY": "Tata",
    "JZ": "Assa-Zag",
    "K": "Tangier",
    "KB": "Tangier",
    "KA": "Asilah",
    "L": "Tétouan",
    "LA": "Larache",
    "LB": "Ksar el-Kebir",
    "LC": "Chefchaouen",
    "LE": "Martil",
    "LF": "Fnideq",
    "LG": "M'diq",
    "M": "El Jadida",
    "MA": "Azemmour",
    "MC": "Sidi Bennour",
    "MD": "Zemamra",
    "N": "Essaouira",
    "O": "Dakhla",
    "OD": "Dakhla",
    "P": "Ouarzazate",
    "PA": "Tinghir",
    "PB": "Zagora",
    "Q": "Khouribga",
    "QA": "Oued Zem",
    "R": "Al Hoceima",
    "RB": "Imzouren",
    "RC": "Targuist",
    "RX": "Bni Bouayach",
    "S": "Nador",
    "SA": "Nador",
    "SH": "Laayoune",
    "SJ": "Smara",
    "SK": "Tarfaya",
    "SL": "Boujdour",
    "T": "Mohammedia",
    "TA": "Benslimane",
    "TK": "Benslimane",
    "U": "Errachida",
    "UA": "Goulmima",
    "UB": "Er-Rich",
    "UC": "Erfoud",
    "UD": "Rissani",
    "V": "Khenifra",
    "VA": "Midelt, Itzer",
    "VM": "M'rirt",
    "W": "Settat",
    "WA": "Berrechid",
    "WB": "Ben Ahmed",
    "X": "Khemisset",
    "XA": "Tifelt",
    "Y": "Kalaat Sraghna",
    "Z": "Taza",
    "ZG": "Guercif",
    "ZH": "Karia Ba Mohamed",
    "ZT": "Taounate",
}


def get_ville(cin):
    if is_valid_cin(cin):
        cin_code = None
        pattern = re.compile(r"^[A-Za-z]+")
        match = pattern.match(cin)
        if match:
            cin_code = match.group(0).upper()
            if cin_code in cin_city:
                return cin_city[cin_code]
            else:
                return None
    return None

In [15]:
check_func = {
    "nom": is_valid_name,
    "prenom": is_valid_name,
    "date_naissance": is_valid_date,
    "cin": is_valid_cin,
    "tel": is_valid_tel,
    "email": is_valid_email,
    "diplome": is_valid_diplome,
    "created": is_valid_created,
    "etat": is_single_digit,
    "viewed": is_single_digit,
    "contacte": is_single_digit,
    "inscrit": is_single_digit,
}

correct_func = {
    "nom": correct_name,
    "prenom": correct_name,
    "date_naissance": correct_date,
    "cin": correct_cin,
    "tel": correct_tel,
    "email": correct_email,
    "diplome": correct_diplome,
    "created": correct_created,
    "etat": correct_single_digit,
    "viewed": correct_single_digit,
    "contacte": correct_single_digit,
    "inscrit": correct_single_digit,
}

## Apply anomaly detection and correction to columns

In [16]:
log_step("Detecting and correcting anomalies in columns")
for column in data.columns:
	if column in check_func :
		anomalies = data[column].apply(check_func[column])
		if anomalies.any():
			for i, value in enumerate(data[column]):
				if not check_func[column](value):
					logging.warning(f"Anomaly detected in column {column} at row {i}: Value '{value}' is incorrect.")
					logging.info(f"Anomalies detected in column {column}. Applying correction function.")
					old_value = value
					new_value = correct_func[column](value)
					data.at[i, column] = new_value
					if old_value != new_value:
						logging.info(f"Column {column}, row {i}: Corrected value: {old_value} -> {new_value}")

			log_success(f"Anomalies corrected in column {column}")
		else :
			logging.info(f"No anomalies detected in column: {column}")

## Interpolating 'created' column

In [17]:
log_step("Interpolating date values in created column")
try :
	data['created'] = pd.to_datetime(data['created'], errors='coerce')
	data['created'] = data['created'].interpolate(method='linear')
	data['created'] = data['created'].dt.round('1s')
	log_success("Interpolating date values in created column")
except Exception as e:
	log_error("Interpolating date values in created column", e)

## Filling 'ville' column from 'cin' column

In [18]:
log_step("Filling column 'ville' based on 'cin' column")
try :
	data['ville'] = data['cin'].apply(get_ville)
	log_success("Filling column 'ville' based on 'cin' column")
except Exception as e:
	log_error("Filling column 'ville' based on 'cin' column", e)


## save the cleaned dataset

In [19]:
log_step("Removing rows with null values")
try :
	data.dropna(inplace=True, ignore_index=True)
	log_success("Removing rows with null values")
except Exception as e:
	log_error("Removing rows with null values", e)

In [20]:
log_step("Saving cleaned data to CSV")
try:
    data.to_csv("cleaned_file.csv", index=False)
    logging.info("Data saved to 'cleaned_file.csv' successfully")
except Exception as e:
    log_error("Saving cleaned data", str(e))

In [21]:
data

,nom,prenom,date_naissance,cin,tel,email,diplome,etablissment,formation,lettre_motivation,etat,viewed,contacte,inscrit,created,ville
0,ER-RAKI,MERYEM,1994-07-21,G664463,+212672619167,meryem.er-raki@outlook.fr,bac+3,UNIVERSITE IBN TOFAI,mi,A Monsieur le directeur de l’École National De...,0,1,1,0,2015-05-20 21:11:59,"Kenitra, Sidi Yahya El Gharb"
1,MEDIOUNE,Badr,1991-04-03,BB80536,+212622431159,badr.medioune@gmail.com,bac+2,ISTA Sidi Moumen,lar,"Monsieur, Titulaire du diplôme technicien spéc...",0,1,1,0,2015-05-23 15:00:16,Casablanca
2,elalami,Alae,1996-01-28,AE136912,+212603787891,el.alamyy.a@gmail.com,bac+2,EST -FES,lar,J’ai choisi de poursuivre mes études en ENSAK ...,0,1,1,0,2015-06-10 02:04:59,Salé
3,MOUZINE,Ismail,1988-02-18,EA135752,+212658717744,mouzine.ismail@gmail.com,bac+2,ISTA I MARRAKECH,GP,Amélioré ma connaissance en production industr...,0,1,1,1,2015-06-13 09:55:53,Ben Guerir
4,oukaidi,fadma,1987-12-05,D733085,+212632065780,fadma.oukaidi@gmail.com,bac+3,FST Settat,MGP,Monsieur Prochainement diplômée d’une licence ...,1,1,0,0,2015-06-16 13:18:23,Meknes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,ELKARKOURI,KHADIJA,1993-03-16,ID46366,+212615042008,elkarkourikhadija123@gmail.com,bac+2,ISGI KHOURIBGA,LRT,j'ai l'honneur de m'accepter,1,1,1,0,2015-10-21 17:28:51,Souk Sebt Ould Nemma
94,hamcha,abdrrahman,1969-12-30,i245113,+212661414268,a.hamcha@africlight.ma,bac+2,ista beni mellal,MGP,bonjour,0,1,1,0,2015-10-28 13:12:27,Beni Mellal
95,bennah,khalil,1993-11-04,OD44737,+212649278532,khalilbennah@gmail.com,bac+3,faculté polydisciplinaire Errachidia,mi,"Après avoir obtenu mon ""diplôme de Licence pro...",0,0,0,0,2015-11-04 14:25:14,Dakhla
96,HIDARA,ADIL,1986-02-07,Q260030,+212615227007,hid.adil@gmail.com,bac+2,ISTA & ENMIG,lil,Très motivé pour continuer mes etude en ce dom...,0,1,0,0,2015-11-11 17:58:45,Khouribga
